In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras import *
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from sklearn.preprocessing import StandardScaler
from sklearn import datasets, svm, metrics
from skimage import io
import os
import matplotlib.pyplot
from matplotlib.pyplot import imread
import skimage.transform
from skimage.transform import resize as imresize
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.preprocessing import image
import csv
import time
import cv2
from PIL import Image


datagen = image.ImageDataGenerator()

TRENING_PUTANJA = './fruits-360/Train'
VALIDACIONA_PUTANJA = './fruits-360/Test'

train_datagen = image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range = 0.1
)

val_datagen = image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range = 0.1,
    validation_split=0.66)

train_generator = train_datagen.flow_from_directory(TRENING_PUTANJA, batch_size=8, target_size=(32, 32))
val_generator = val_datagen.flow_from_directory(VALIDACIONA_PUTANJA, batch_size=8, target_size=(32, 32), subset='validation')
test_generator = val_datagen.flow_from_directory(VALIDACIONA_PUTANJA, batch_size=8, target_size=(32, 32), subset='training')


Found 7695 images belonging to 15 classes.
Found 1694 images belonging to 15 classes.
Found 882 images belonging to 15 classes.


In [9]:
tf.disable_v2_behavior()

model = Sequential()

model.add(Conv2D(32,(3,3),input_shape=(32,32,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(128,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(15))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(),metrics=['accuracy'])
history = model.fit_generator(train_generator, epochs=6, steps_per_epoch=train_generator.n, 
                              validation_data=val_generator, validation_steps=val_generator.n)

Epoch 1/6
7695/7695 [==============================] - 678s 88ms/step - loss: 0.2714 - acc: 0.9198 - val_loss: 0.6710 - val_acc: 0.8983
Epoch 2/6
7695/7695 [==============================] - 614s 80ms/step - loss: 0.0851 - acc: 0.9787 - val_loss: 0.0235 - val_acc: 0.9937
Epoch 3/6
7695/7695 [==============================] - 546s 71ms/step - loss: 0.0491 - acc: 0.9884 - val_loss: 0.0773 - val_acc: 0.9823
Epoch 4/6
7695/7695 [==============================] - 533s 69ms/step - loss: 0.0446 - acc: 0.9906 - val_loss: 0.0043 - val_acc: 0.9985
Epoch 5/6
7695/7695 [==============================] - 3003s 390ms/step - loss: 0.0364 - acc: 0.9928 - val_loss: 0.0042 - val_acc: 0.9989
Epoch 6/6
7695/7695 [==============================] - 534s 69ms/step - loss: 0.0303 - acc: 0.9943 - val_loss: 0.0429 - val_acc: 0.9899


In [16]:
predict = model.predict_generator(test_generator, steps=test_generator.n)


In [17]:
y_pred = np.argmax(predict, axis = -1)
print(y_pred)


[9 4 2 ... 0 1 6]


In [18]:
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in y_pred]